# Import libraries

In [ ]:
import os
import glob
import pylab
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Functions

In [ ]:
style_param = {'legend.fontsize': '20',
         'xtick.direction' : 'in',  
         'ytick.direction' : 'in', 
         'xtick.top' : True, 
         'figure.figsize': (10,6),
         'axes.labelsize': '26',
         'axes.titlesize':'26',
         'xtick.labelsize':'20',
         'ytick.labelsize':'20',
         'xtick.major.pad':'16',
         'ytick.major.pad':'16'}
    
pylab.rcParams.update(style_param)

# Prepare folders

In [ ]:
os.makedirs('results',exist_ok=True)

In [ ]:
os.makedirs('tmp',exist_ok=True)

# Read DATA

In [ ]:
df_list=[]
df_list_err=[]
df_list_coe=[]

for f in glob.glob(os.path.join('data','*', '*.csv')):
    art = f.split(os.path.sep)[-2]
    filename=f.split(os.path.sep)[-1]
    df_tmp=pd.read_csv(f, 
                       names=['x','y'],
                       converters = { 'x' : lambda s : float(s.replace(',', '.')), 'y' : lambda s : float(s.replace(',', '.')), },
                       delimiter=';')
    if filename.startswith('err'):
        #energy in [MeV]       
        energy=float(filename[3:-7].replace(',', '.'))
        df_tmp_err = pd.DataFrame()
      
        up_candidates = df_tmp.y[0::2].values
        down_candidates = df_tmp.y[1::2].values
        df_tmp_err['sf_err_up'] = [max(up, down) for up, down in zip(up_candidates, down_candidates)]
        df_tmp_err['sf_err_down'] = [min(up, down) for up, down in zip(up_candidates, down_candidates)]
        
        
        
        #convert energy [MeV]->[keV] 
        df_tmp_err['energy']=energy*1000
        df_tmp_err['article']=art
        df_tmp_err['nr']=np.arange(df_tmp_err.energy.size)
        df_list_err.append(df_tmp_err)
            
    elif filename.startswith("coe"):
        #energy in [MeV]
        energy=float(filename[3:-7].replace(',', '.'))
        df_tmp_coe = pd.DataFrame()
        df_tmp_coe["alpha"]=df_tmp.x.loc[[0]]
        df_tmp_coe["beta"]=df_tmp.x[1]
        df_tmp_coe["alpha_err"]=df_tmp.x[2]
        df_tmp_coe["beta_err"]=df_tmp.x[3]
        df_tmp_coe["LET"]=df_tmp.x[4]
        df_tmp_coe['article']=art
        #convert energy [MeV]->[keV] 
        df_tmp_coe['energy']=int(energy*1000)
        df_list_coe.append(df_tmp_coe)

    else: 
        #energy in [MeV]
        energy=float(filename[0:-7].replace(',', '.'))
        #convert energy [MeV]->[keV] 
        df_tmp['energy']=int(energy*1000)
        df_tmp['article']=art       
        #add a column to number each row for each experiment -> needed for concatenation
        df_tmp['nr']=np.arange(df_tmp.energy.size)
       
        df_list.append(df_tmp)

In [ ]:
result_err=pd.concat(df_list_err, axis=0)
result_err.replace(',', '.', inplace=True)
result_err['article'] = result_err.article.astype('category')

result_coe=pd.concat(df_list_coe, axis=0)
result_coe.replace(',', '.', inplace=True)
result_coe['article'] = result_coe.article.astype('category')

result=pd.concat(df_list, axis=0)
result.replace(',', '.', inplace=True)
result.rename(index=str, columns={"x": "dose", "y": "sf"}, inplace=True)
result['article'] = result.article.astype('category')

# Concatenation of DataFrames

In [ ]:
result.reset_index(drop=True,inplace=True)
result_coe.reset_index(drop=True,inplace=True)
result_err.reset_index(drop=True,inplace=True)

In [ ]:
result_data = pd.merge(result, result_coe,on=["article","energy"])
result_data['article'] = result_data.article.astype('category')

In [ ]:
result_data.set_index(['article','energy','nr'],inplace=True)
result_err.set_index(['article','energy','nr'],inplace=True)

In [ ]:
result_all = pd.merge(result_data,  result_err,left_index = True, right_index=True)

In [ ]:
df2 = result_all.groupby(["article",'energy']).max()

In [ ]:
for name, group in df2.groupby('article'):
        for energy, data in group.groupby('energy'):
            df2.loc[(name,energy),'alpha_ref'] = group.loc[name].loc[0].alpha
            df2.loc[(name,energy),'alpha_ref_err'] = group.loc[name].loc[0].alpha_err
            df2.loc[(name,energy),'beta_ref'] = group.loc[name].loc[0].beta
            df2.loc[(name,energy),'beta_ref_err'] = group.loc[name].loc[0].beta_err

In [ ]:
#beta = 0 ? 
for name, group in df2.groupby('article'):
        for energy, data in group.groupby('energy'):
            df2.loc[(name, energy),'alpha_alpharef_err'] = data.alpha_err.values/data.alpha_ref.values -data.alpha.values*data.alpha_ref_err.values/(data.alpha_ref.values*data.alpha_ref.values)
            df2.loc[(name, energy),'beta_betaref_err'] = data.beta_err.values/data.beta_ref.values -data.beta.values*data.beta_ref_err.values/(data.beta_ref.values*data.beta_ref.values)

# using Original Data

In [ ]:
bb0,bb1,err,err0 =[],[],[],[]

for name, group in df2.groupby('article'):
        for energy, data in group.groupby('energy'):

            bb0.append(data.beta_ref.values[0])
            bb1.append(data.beta.values[0])
            err.append(data.beta_err.values)
            err0.append(data.beta_ref_err.values)
err = np.concatenate(err, axis=0 )
err0 = np.concatenate(err0, axis=0 )

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))

ax.errorbar(bb0,bb1,
             yerr=err,
             xerr=err0,
             zorder=2, 
             fmt='o',color='k',capsize=5)

ax.set_ylabel(r'$\beta\;[Gy^{-2}]$'+'\n')
ax.set_xlabel('\n'+r'$\beta_{ref} \;[Gy^{-2}]$')

ax.set_xlim(0,None)
ax.set_ylim(0,None)
ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
tmp=pd.DataFrame(list(zip(bb0,bb1,err,err0)),
              columns=['beta_ref','beta', 'bera_ref_err','beta_err'])
tmp.to_csv("results/figure_4_2.csv")
fig.savefig(fname="results/figure_4_2.png",dpi= 700)

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))

ax.errorbar(df2.LET.values[df2.LET.values>0],
             df2.alpha[df2.LET.values>0].values/df2.alpha_ref[df2.LET.values>0].values,
             yerr=df2.alpha_alpharef_err[df2.LET.values>0],
             capsize=4,
             fmt='o',color='k', label='both limits (default)')


ax.set_xlabel('\n'+r'$LET \; [keV \cdot\mu m^{-1}]$')
ax.set_ylabel(r'$\alpha/ \alpha_{ref}\;[-]$'+'\n')

ax.set_xlim(0,32)

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
tmp=pd.DataFrame(list(zip(df2.LET.values[df2.LET.values>0],
                          df2.alpha[df2.LET.values>0].values/df2.alpha_ref[df2.LET.values>0].values,
                          df2.alpha_alpharef_err[df2.LET.values>0])),
              columns=['LET','alpha/alpha_ref', 'alpha_err'])
tmp.to_csv("results/figure_4_1.csv")

fig.savefig(fname="results/figure_4_1.png",dpi= 700)

In [ ]:
fig,ax = plt.subplots(figsize = (10,6))

ax.errorbar(df2.LET[(df2.LET.values>0) & (df2.beta_ref>0)].values,
             df2.beta[(df2.LET.values>0) & (df2.beta_ref>0)].values/df2.beta_ref[(df2.LET.values>0.) & (df2.beta_ref>0)].values,
             yerr=df2['beta_betaref_err'][(df2.LET.values>0) & (df2.beta_ref>0)],
             capsize=4,
             fmt='o',color='k')

ax.set_xlabel('\n'+r'$LET \; [keV \cdot\mu m^{-1}]$')
ax.set_ylabel(r'$\beta / \beta_{ref}\;[-]$'+'\n')
ax.axhline(1,c='k',ls='--')

ax.set_xlim(0,32)
ax.set_ylim(-1,5)

ax.grid()
ax.minorticks_on()
ax.grid(which='minor', linestyle=':', linewidth='0.2', color='k')

In [ ]:
tmp=pd.DataFrame(list(zip(df2.LET[(df2.LET.values>0) & (df2.beta_ref>0)].values,
                           df2.beta[(df2.LET.values>0) & (df2.beta_ref>0)].values/df2.beta_ref[(df2.LET.values>0.) & (df2.beta_ref>0)].values,
                           df2.beta_betaref_err[(df2.LET.values>0) & (df2.beta_ref>0)])),
              columns=['LET','beta/beta_ref', 'beta_err'])
tmp.to_csv("results/figure_4_3.csv")

fig.savefig(fname="results/figure_4_3.png",dpi= 700)

# Save temporary files

In [ ]:
fname = os.path.join('tmp','rawdata.h5')
result_all.to_hdf(fname, 'data', format='table')